In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import classifier

In [4]:
import nltk
import torch
import torchtext.vocab as vocab
import torch.optim as optim
import torch.autograd as autograd
import torch.nn as nn

In [5]:
glove = vocab.GloVe(name='6B', dim=50)

# preprocess data

In [6]:
subset_train = 0.1

In [7]:
datapath = '../data/'

In [8]:
train = pd.read_csv(datapath + 'train.csv')
test = pd.read_csv(datapath + 'test.csv')
sub = pd.read_csv(datapath + 'sample_submission.csv')
train = train.iloc[:int(len(train) * subset_train), :]
test = test.fillna('unknown')
comment_types = train.columns[2:].values

In [9]:
lang = classifier.Language()

In [10]:
lang.process_sentences(train.comment_text.values)

In [11]:
init_embedding = classifier.create_init_embedding(lang, glove)

In [12]:
model = classifier.NER_LSTM(lang.vocab_size,
                           embedding_dim=50,
                           hidden_dim=32,
                            linear_dim=64, 
                           n_layers=1,
                            dropout=.2,
                           init_embedding=init_embedding,
                           bidirectional=True)

In [13]:
loss = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
trainer = classifier.Trainer(model,
                             optimizer,
                             loss,
                             lang.encoded_sentences,
                             train.loc[:, comment_types].values.astype(int),
                             batch_size = 32)

In [15]:
trainer.train(1)

Epoch: 0    loss: 0.1927, val: 0.1474
